In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(master="local", appName="transformacionesYAcciones")

In [3]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [4]:
rdd1.collect()

[1, 2, 3]

In [5]:
sc

<SparkContext master=local appName=transformacionesYAcciones>

In [6]:
!ls

 codeExample.py   data	'Transformaciones y acciones.ipynb'   Untitled.ipynb


In [7]:
equiposOlimpicosRDD = sc.textFile("./data/paises.csv")\
                        .map(lambda line: line.split(",")) #como es csv esta separado por comas

In [8]:
equiposOlimpicosRDD.take(5) 

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

In [9]:
# sc.stop()

In [10]:
equiposOlimpicosRDD.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [11]:
# Es preferible trabajar con tuplas en vez de listas
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

In [12]:
# GroupBy
# El groupby se hace considerando el primero
equiposOlimpicosRDD.map(lambda x: (x[2],x[1])) \
                        .groupByKey() \
                        .mapValues(len) \
                        .take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [13]:
equiposOlimpicosRDD.map(lambda x: (x[2],x[1])) \
                        .groupByKey() \
                        .mapValues(list) \
                        .take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [14]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l: "ARG" in l)
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [15]:
# EL numero entrega los milisegundos
equiposOlimpicosRDD.countApprox(60)

1185

In [27]:
! ls data

data.csv	 deportista.csv       juegos.csv	     resultados.csv
deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv


In [28]:
deportistaOlimpicoRDD = sc.textFile("./data/deportista.csv") \
    .map(lambda l: l.split(","))
deportistaolimpicoRDD2 = sc.textFile("./data/deportista2.csv") \
    .map(lambda l: l.split(","))
# Concatenacion
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaolimpicoRDD2)

In [29]:
deportistaOlimpicoRDD.count()

135572

In [30]:
deportistaOlimpicoRDD.top(4)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967'],
 ['99998', 'Robert John Bob Renney', '1', '21', '178', '90', '66'],
 ['99997', 'Thomas Renner', '1', '24', '183', '86', '71']]

In [31]:
equiposOlimpicosRDD.top(4)

[['id', 'equipo', 'sigla'],
 ['999', 'Stella-2', 'NOR'],
 ['998', 'State VI', 'CAN'],
 ['997', 'Starlight III', 'GBR']]

In [32]:
deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x: [x[0],x[2]])) \
    .take(6)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN')),
 ('199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'CHN'))]

In [34]:
deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x: [x[0],x[2]])) \
    .takeSample(False, 6, 25)

[('362', (['131505', 'Steven Woodburn', '1', '24', '185', '90'], 'FRA')),
 ('967', (['13626', 'Jill Brresen', '2', '22', '170', '57'], 'RSA')),
 ('482', (['44299', 'Gumundur Gumundsson', '1', '23', '174', '77'], 'ISL')),
 ('970', (['68062', 'Lee MinHui', '2', '28', '174', '65'], 'KOR')),
 ('794', (['92442', 'Luis Paz Zoldan', '1', '19', '187', '82'], 'PER')),
 ('413', (['26822', 'Jared Mark Deacon', '1', '24', '185', '77'], 'GBR'))]

In [39]:
resultado = sc.textFile("./data/resultados.csv") \
    .map(lambda l : l.split(","))

In [40]:
resultado.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['1', 'NA', '1', '39', '1'],
 ['2', 'NA', '2', '49', '2'],
 ['3', 'NA', '3', '7', '3'],
 ['4', 'Gold', '4', '2', '4']]

In [41]:
resultadoGanador = resultado.filter(lambda l : "NA" not in l[1])
resultadoGanador.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [44]:
deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
    .join(equiposOlimpicosRDD.map(lambda x: [x[0],x[2]])) \
    .join(resultadoGanador.map(lambda y: [y[2],y[1]])) \
    .takeSample(False, 6, 25)

[('507',
  ((['18909', 'Enrico Castelli', '1', '27', '0', '0'], 'ITA'), 'Silver')),
 ('705',
  ((['99600', 'John Philip Ernest Marie Flip Regout', '1', '21', '0', '0'],
    'NED'),
   'Silver')),
 ('399',
  ((['15937', 'Magdalena Brzeska Peschel Sabolocka ', '2', '18', '173', '48'],
    'GER'),
   'Silver')),
 ('705',
  ((['13408',
     'Anna Johanna Geertruida Maria Annie Borckink',
     '2',
     '24',
     '0',
     '0'],
    'NED'),
   'Bronze')),
 ('705',
  ((['48310', 'Johannes Hendricus Heuckelbach', '1', '27', '0', '0'], 'NED'),
   'Gold')),
 ('619',
  ((['59062', 'Noraseela Mohd Khalid', '2', '32', '166', '55'], 'MAS'),
   'Gold'))]

In [49]:
deportistaPaises = deportistaOlimpicoRDD.map(lambda l :[l[-1],l[:-1]]) \
                    .join(equiposOlimpicosRDD.map(lambda x: [x[0], x[2]])) 
deportistaPaises.take(3)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN'))]

In [50]:
deportistaPaises.join(resultadoGanador).take(6)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold')),
 ('74', ((['130', 'Tural Abbasov', '1', '18', '182', '76'], 'AZE'), 'Gold')),
 ('74', ((['131', 'Tran Abbasova', '2', '33', '159', '53'], 'AZE'), 'Gold')),
 ('74',
  ((['335', 'Abdulqdir Abdullayev', '1', '28', '188', '91'], 'AZE'), 'Gold')),
 ('74',
  ((['336', 'Arif Yadulla Abdullayev', '1', '27', '164', '60'], 'AZE'),
   'Gold'))]

### Operaciones numericas

In [61]:
valoresMedallas = { "Gold":7, "Silver":5 , "Bronze": 4 }

In [62]:
paisesMedallas = deportistaPaises.join(resultadoGanador)

In [63]:
paisesMedallas.map(lambda x: [x[1][0][-1],x[1][1]]).take(5)

[['AZE', 'Gold'],
 ['AZE', 'Gold'],
 ['AZE', 'Gold'],
 ['AZE', 'Gold'],
 ['AZE', 'Gold']]

In [64]:
paisesMedallas = paisesMedallas.map(lambda x: [x[1][0][-1],valoresMedallas[x[1][1]]])

In [66]:
from operator import add
conclusion = paisesMedallas.reduceByKey((add)).sortBy(lambda x: x[1], ascending=False)

In [67]:
conclusion.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]